# Art style transfer

Código basado en el paper [A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576).

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import cv2
import numpy as np


def load_img(base):
    return cv2.cvtColor(cv2.imread(base), cv2.COLOR_BGR2RGB)

style_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

content_layers = ['block4_conv2']

model = VGG16(include_top=False)

model.summary() 

#def preprocess_input(img):
#    return img # / 255

In [ ]:
base = load_img('tandil.jpg')
img_rows = 400
shape = (img_rows, int(base.shape[1] / base.shape[0] * img_rows), 3)

base = cv2.resize(base, (shape[1], shape[0]))


style = load_img('style.jpg')
style = cv2.resize(style, (shape[1], shape[0]))

base = np.expand_dims(preprocess_input(base), 0)

style = np.expand_dims(preprocess_input(style), 0)



In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

outs = []

i = Input((None, None, None))
ant = i
inx = 0

for l in model.layers[1:]:
    if inx == len(style_layers):
        break
    ant = l(ant)
    if l.name == style_layers[inx]:
        inx += 1
        outs.append(ant)

modelStyle = Model(i, outs)


outs = []

i = Input((None, None, None))
ant = i
inx = 0

for l in model.layers[1:]:
    if inx == len(content_layers):
        break
    ant = l(ant)
    if l.name == content_layers[inx]:
        inx += 1
        outs.append(ant)

modelContent = Model(i, outs)
modelContent.summary()

In [ ]:
base_features = [x for x in modelContent.predict(base)]
style_features = [x[0,...] for x in modelStyle.predict(style)]
#style_features1 = [x[0, ...] for x in modelStyle.predict(style)]

In [ ]:
#style_features[0] == style_features1[0]
print(style_features[0].shape)
print(base_features[0].shape)

In [ ]:
import tensorflow as tf


def gram_mat(m):
    m = tf.transpose(m, (2, 0, 1))
    return tf.matmul(m, m, transpose_b=True)

def gram_mat(m):
    m = tf.transpose(m, (2, 0, 1))
    m = tf.reshape(m, tf.stack([-1, tf.reduce_prod(m.shape[1:], None, False)]))
    return tf.matmul(m, m, transpose_b=True)


In [ ]:
style_features = [gram_mat(x) for x in style_features]
#base_features = [tf.constant(x) for x in base_features]

In [ ]:
style_features[0].shape

In [ ]:
def loss(x, a=1, b=1, c=1):
    x = tf.reshape(x, (1,) + shape)
    fc = modelContent(x)
    fs = [gram_mat(l[0]) for l in modelStyle(x)]

    loss_c = 0
    for fci, fcb in zip(fc, base_features):
        loss_c += tf.reduce_sum(tf.math.squared_difference(fci, fcb))#/ (4 * fci.shape[0] * fci.shape[1] * fci.shape[2]) / len(fc)
    
    #loss_c = loss_c / (4 * shape[0] * shape[1] * shape[2])
    loss_s = 0
    #for fsi, fss in zip(fs, style_features):
    #    loss_s += tf.reduce_sum(tf.math.squared_difference(fsi, fss)) / (4 * fsi.shape[0] ** 2 * fsi.shape[1] ** 2 * fsi.shape[2] ** 2) / len(fs)
    for fsi, fss in zip(fs, style_features):
        loss_s += tf.reduce_sum(tf.math.squared_difference(fsi, fss)) / (4 * fsi.shape[0] ** 2 * fsi.shape[1] ** 2) / len(fs)


    ai = tf.math.squared_difference(x[:, :-1, :-1, :], x[:, 1:, :-1, :])
    bi = tf.math.squared_difference(x[:, :-1, :-1, :], x[:, :-1, 1:, :])
    var_loss = tf.reduce_sum(tf.pow(ai+bi, 1.25))
    var_loss = tf.cast(var_loss, tf.float32)
    
    return a * loss_c + b * loss_s + c * var_loss

In [ ]:
a = np.random.rand(shape[0] * shape[1] * shape[2]) * 150

def to_image(x): 
    x = np.reshape(x, shape)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    #x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

'''def to_image(x): 
    x = np.reshape(x, shape)# * 255
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x'''

loss(a, a=1, b=0, c=0)

In [ ]:
from google.colab.patches import cv2_imshow

cv2_imshow(to_image(a))

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt


w = tf.random.uniform(shape=[shape[0] * shape[1] * shape[2]], minval=-150, maxval=150)
ciclos = 100
lr = 0.001#0.01 
momentum = 0.9
errors = []
for i in tqdm(range(ciclos)):
    with tf.GradientTape() as g:
        g.watch([w])
        l = loss(w,a=1500, b=10, c=0) 
        errors.append(l.numpy())
    gw = tf.clip_by_value(g.gradient(l, w), -10000, 10000)#mejor
    w = tf.clip_by_value(w - lr * gw, -200, 200) #mejor
    #cv2_imshow(to_image(w.numpy()))
    if i > 40:
        lr = 0.0001

print('Errores a medida que se actualiza el valor de w')
plt.plot(errors)
plt.show()


In [ ]:
cv2_imshow(to_image(w.numpy()))

In [ ]:
w = tf.random.uniform(shape=[shape[0] * shape[1] * shape[2]], minval=-150, maxval=150)
ciclos = 5
lr = 0.001 
cant = 0

In [ ]:
cv2_imshow(to_image(w.numpy()))

In [ ]:
for i in tqdm(range(ciclos)):
    with tf.GradientTape() as g:
        g.watch([w])
        l = loss(w,a=500, b=2, c=0)
        errors.append(l.numpy())
    gw = tf.clip_by_value(g.gradient(l, w), -10000, 10000)
    w = tf.clip_by_value(w - lr * gw, -200, 200)
cv2_imshow(to_image(w.numpy()))